In [9]:
#Explore XRP Price Trends
from matplotlib import pyplot as plt
import pandas as pd
from ts_fun import build_lag_data, corrcof, delta, gen_ols, lag_ols, linefit,evaluate_ts, train_test_split, data_set_sample, predict, sample
from tslibs import measure
import data_download as dd
import keras as k
import numpy as np

def update(sym="XRP-USD"):
    d1,d2 = dd.to_csv(sym)

    d1.to_csv(f"data/{sym}.csv")
    d2.to_csv(f"data/{sym}-price.csv")
    print(f"{sym} files written.")

def load_from_file(sym="XRP-USD"):
    price = pd.read_csv(f"data/{sym}-price.csv",index_col="time",parse_dates=True)
    daily = pd.read_csv(f"data/{sym}.csv",index_col="time",parse_dates=True)

    return price,daily 

#VARS
UPDATE_DATASETS = True
SYMBOL = "XRP-USD"
GBP_USD = 1.22

In [10]:
# Optional - to update datasets
if UPDATE_DATASETS:
    update(sym="XRP-USD")
    update(sym="BTC-USD")
    update(sym="ETH-USD")
    update(sym="BCH-USD")


XRP-USD files written.
BTC-USD files written.
ETH-USD files written.
BCH-USD files written.


In [11]:
#Pipelines
def hohc_lolc(daily):
    ho = daily["high"]-daily["open"]
    lo = daily["low"] - daily["open"]
    hc = daily["high"] - daily["close"]
    lc = daily["low"] - daily["close"]

    return ho, hc, lo, lc

def n(x):
    
    return x/x.max()

def ntrain_set_predict(daily):
    low = daily["low"]
    X = [
        n(daily["open"])[:-1],
        n(daily["close"])[:-1],
        n(low[:-1]),
        n(daily["high"])[:-1],
        n(daily["volume"])[:-1]
    ]
    Y = [int(x > 0) for x in np.diff(low,1)]
    

    return np.array(X).transpose(),np.array([Y])

def train_set_predict(daily):
    low = daily["low"]
    X = [
        daily["open"][:-1],
        daily["close"][:-1],
        low[:-1],
        daily["high"][:-1],
        daily["volume"][:-1]
    ]
    Y = [int(x > 0) for x in np.diff(low,1)]
    

    return np.array(X).transpose(),np.array(Y)



In [12]:
#Models 
def nn_model(f_len=5):
    m = k.models.Sequential()
    m.add(k.layers.Dense(f_len,activation='sigmoid'))
    m.add(k.layers.Dense(f_len, activation='relu'))
    m.add(k.layers.Dense(1,activation='sigmoid'))
    

    m.compile(optimizer='adam',loss='binary_crossentropy', metrics=["accuracy","binary_crossentropy"])

    return m


In [13]:

    
# Load data
price, daily = load_from_file(sym=SYMBOL) 

In [14]:

X, Y = train_set_predict(daily)
print(X.shape, Y.shape)


(2835, 5) (2835,)


In [15]:

m = nn_model(len(X))
X_train,Y_train,X_test,Y_test = data_set_sample(X,Y)
#print(X_train)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)


(2268, 5) (567, 5) (2268,) (567,)


In [16]:

m.fit(X_train,Y_train, epochs=10,batch_size=10,verbose=1)
m.evaluate(X_test,Y_test,verbose=0)



Epoch 1/10
227/227 [==============================] - 14s 62ms/step - loss: 0.8385 - accuracy: 0.5084 - binary_crossentropy: 0.8385
Epoch 2/10
227/227 [==============================] - 15s 64ms/step - loss: 0.6931 - accuracy: 0.5234 - binary_crossentropy: 0.6931
Epoch 3/10
227/227 [==============================] - 14s 61ms/step - loss: 0.6929 - accuracy: 0.5225 - binary_crossentropy: 0.6929
Epoch 4/10
227/227 [==============================] - 14s 63ms/step - loss: 0.6927 - accuracy: 0.5234 - binary_crossentropy: 0.69271s - loss: 0.6929 - accura
Epoch 5/10
227/227 [==============================] - 14s 63ms/step - loss: 0.6924 - accuracy: 0.5234 - binary_crossentropy: 0.6924
Epoch 6/10
227/227 [==============================] - 15s 64ms/step - loss: 0.6922 - accuracy: 0.5234 - binary_crossentropy: 0.6922
Epoch 7/10
227/227 [==============================] - 14s 63ms/step - loss: 0.6922 - accuracy: 0.5234 - binary_crossentropy: 0.6922
Epoch 8/10
227/227 [==============================

[0.691150963306427, 0.5343915224075317, 0.691150963306427]

In [17]:

# process result

print(np.array([X[-1]]).shape)

print(m.predict(np.array([X[-1]]),verbose=1))
print(X[-1])

(1, 5)
1/1 [==============================] - 0s 2ms/step
[[0.5207837]]
[3.23284000e-01 3.28626007e-01 3.20152014e-01 3.36062998e-01
 1.24758200e+09]
